In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

import time
import random
import re

import pandas as pd

In [2]:
result = {}
page = 0
count = 0

In [3]:
def parse_cian_with_defined_params(rooms_count=1):
    driver = webdriver.Chrome()
    #оставляем слип на открытие страницы
    time.sleep(2)
    result = {}
    page = 1
    count = 0
    driver.get(f'https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p={page}&region=1&room{rooms_count}=1')
    time.sleep(2)
    while True:
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         time.sleep(random.uniform(1,2))
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source,'lxml')

        #тут карточки всех объектов недвиги
    #     for i in soup.find_all('div','_93444fe79c--card--ibP42 _93444fe79c--wide--gEKNN'):
        for i in soup.find_all('div','_93444fe79c--card--ibP42'):
            # i - карточка отдельного объекта
            result[count] = {}

            link = i.find('a', '_93444fe79c--link--eoxce')
            result[count]['link'] = link['href']

            title = i.find('span',{'data-mark':'OfferTitle'})
            result[count]['title'] = title.text

            try:
                subtitle = i.find('span',{'data-mark':'OfferSubtitle'})
                result[count]['subtitle'] = subtitle.text
            except:
                pass

            try:
                deadline = i.find('span', {'data-mark':'Deadline'})
                result[count]['deadline'] = deadline.text
            except:
                pass

            try:
                metro = i.find('a','_93444fe79c--link--BwwJO')
                result[count]['metro'] = metro.text
                metro_remote = i.find('div','_93444fe79c--remoteness--q8IXp')
                result[count]['metro_remote'] = metro_remote.text

            except:
                pass

            geo = i.find('div','_93444fe79c--labels--L8WyJ')
            result[count]['geo'] = geo.text

            main_price = i.find('span',{'data-mark':'MainPrice'})
            result[count]['main_price'] = int(re.sub(' ', '',main_price.text[:-2]))
            if main_price.text[-1] == '₽':
                currency = 'rouble'
            else:
                currency = 'other'
            result[count]['currency'] = currency

            price_for_sq_m = i.find('p',{'data-mark':'PriceInfo'})
            result[count]['price_for_sq_m'] = int(re.sub(' ', '',price_for_sq_m.text[:-5]))

            count += 1

        if soup.find_all('div',{'data-name':'Suggestions'}) == []:
            page += 1
            driver.get(f'https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p={page}&region=1&room{rooms_count}=1')
#             time.sleep(random.uniform(1,3))
            time.sleep(1)
        else:
            print('цикл остановлен')
            driver.quit()
            return result


In [4]:
%%time
room_1 = parse_cian_with_defined_params(rooms_count=1)
room_2 = parse_cian_with_defined_params(rooms_count=2)
room_3 = parse_cian_with_defined_params(rooms_count=3)
room_4 = parse_cian_with_defined_params(rooms_count=4)
room_5 = parse_cian_with_defined_params(rooms_count=5)
room_6 = parse_cian_with_defined_params(rooms_count=6)
room_stud = parse_cian_with_defined_params(rooms_count=9)

цикл остановлен


KeyboardInterrupt: 

In [5]:
***

SyntaxError: invalid syntax (2938429678.py, line 1)

In [ ]:
result = pd.concat([
    pd.DataFrame(room_1).T,
    pd.DataFrame(room_2).T,
    pd.DataFrame(room_3).T,
    pd.DataFrame(room_4).T,
    pd.DataFrame(room_5).T,
    pd.DataFrame(room_6).T,
    pd.DataFrame(room_stud).T
])
result

In [ ]:
result = result.drop_duplicates().reset_index(drop=True)
result

In [ ]:
result.drop('link', axis=1).drop_duplicates()

In [ ]:
result[result['main_price'] < 10_000_000].sort_values('price_for_sq_m').head(20)

In [ ]:
result.to_json('cian.json')

In [ ]:
***